# Documentation

# Question 1 

I chose Simmulated Annealing as a stochastic optimiser for Task 3 

# Question 2 Why did you choose this optimiser?

The spring relies on multiple variables making the graphical image of what the data might look like very complex, this means 
we cannot just choose a simple algorithm to optimise the problem and a stochastic approach would be beneficial.
Simulated annealing allows distortion to be added to the data set to avoid getting stuck at a local minima,
this normally allows a good optimal to be found. Simulated annealing works better with continuous function approximation 
than GA algorithms. As f(x) is a continuous function this makes simulated annealing the better option that should be implemented
to solve this problem. [99 words]

# Question 3 What constraint handling method(s) are you using and why?

To handle the inequality constraints of g1 to g4 and to keep all the simulated annealing points within the feasible space I
have used death penalties. Any combinations of values (x1, x2, x3) that do not pass g1 to g4 are rejected as these would 
produce an f(x) outside of the feasible region and take up more computational power than necessary.


In [1]:
import numpy as np
import math
import random
import matplotlib as mpl
import matplotlib.pyplot as plt 

#make the random number predicatable 
np.random.seed(250599)

In [2]:
arr_X = np.array([1, 2, 3])

# Task 1 implementation of f(x)
Question 1

: the diameter of the wire (x0), 
the mean of the diameter of coil(x1)
and the number of active coils (x2)

In [3]:
def f(x, fCount=[0]):
    fun = float(x[0] ** 2)*x[1]*(2 + x[2])
#     print(fun)
    fCount[0]+=1
    return [fun, fCount[0]]

def g1(x, g1Count=[0]):
    g1Count[0]+=1
    fun = float(1 - (((x[1]**3)*x[2])/(71785*(x[0]**4))))
    return [fun <= 0, g1Count[0]]
    
def g2(x, g2Count=[0]):
    fun = (   (((4*x[1]**2) - (x[0]*x[1]))/(12566*((x[1]*(x[0]**3))-(x[0]**4)))) + (1/(5108*(x[0]**2)))  - 1 )
    g2Count[0]+=1
    return[fun <= 0, g2Count[0]]

    
def g3(x, g3Count=[0]):
    fun = (1 - (140.45*x[0])/((x[1]**2)*x[2]))
    g3Count[0]+=1
    return[fun <= 0, g3Count[0]]

    
def g4(x, g4Count=[0]):
    fun = (  ((x[0] + x[1])/1.5)-1  )
    g4Count[0]+=1
    return[fun <= 0, g4Count[0]]

    

In [4]:
print(f(arr_X))
print(f(arr_X))
print(f(arr_X))

print (g1(arr_X))
print (g1(arr_X))
print (g1(arr_X))

print (g2(arr_X))
print (g2(arr_X))
print (g2(arr_X))

print (g3(arr_X))
print (g3(arr_X))
print (g3(arr_X))

print (g4(arr_X))
print (g4(arr_X))
print (g4(arr_X))  



[10.0, 1]
[10.0, 2]
[10.0, 3]
[False, 1]
[False, 2]
[False, 3]
[True, 1]
[True, 2]
[True, 3]
[True, 1]
[True, 2]
[True, 3]
[False, 1]
[False, 2]
[False, 3]


# Task 2 - Random Search

In [5]:
wire_lower = 0.05 #lower bound for diameter of the wire
wire_upper = 2 #upper bound for diameter of the wire

coil_lower = 0.25 #lower bound for diameter of coil
coil_upper = 1.3 #upper bound for diameter of the coil

coil_num_lower = 2 #lower bound for number of active coils
coil_num_upper = 15 #upper bound for number of active coils

In [6]:
def generate_random_array():
    #generate random number 
    x0 = np.random.uniform(low=wire_lower, high=wire_upper) #hightest + 1
    x1 = np.random.uniform(low=coil_lower, high=coil_upper) #hightest + 1
    x2 = np.random.uniform(low=coil_num_lower, high=coil_num_upper) #hightest + 1
    
    return np.array([x0, x1, x2])
    

In [7]:
def is_feasible(array):
    feasible = False
    g1_arr=g1(array)
    g2_arr=g2(array)
    g3_arr=g3(array)
    g4_arr=g4(array)

#     print(g1_arr)
#     print(g2_arr)
#     print(g3_arr)
#     print(g4_arr)

    if g1_arr[0] and g2_arr[0] and g3_arr[0] and g4_arr[0] :
        feasible = True
    else:
        feasible = False
    return feasible
    

In [8]:
def generate_feasible_random_array():
    feasible = False
    while not feasible:
        array = generate_random_array()
        feasible = is_feasible(array)
    return array

In [9]:
def random_search(iteration=3000):
    
    best_fx = [1000,None]
    
    for x in range(iteration):
        arr_x = generate_feasible_random_array()
        
        fx = f(arr_x)
        if fx[0] < best_fx[0]:
            best_x = np.array([arr_x[0], arr_x[1], arr_x[2]])
            best_fx = fx

    return [best_x, best_fx]

In [10]:
random_search()

KeyboardInterrupt: 

# Task 3 - Simulated Annealing

In [40]:
def distort(array):
    x0 = array[0]
    x1 = array[1]
    x2 = array[2]
    
    x0_new = 1000 #not feasible to begin with
    x1_new = 1000 #not feasible to begin with
    x2_new = 1000 #not feasible to begin with
    
    while not is_feasible(np.array([x0_new, x1, x2])):
        x0_new = np.random.normal(x0, 1)
        x0_clipped = np.clip(x0_new, wire_lower, wire_upper)
    
    while not is_feasible(np.array([x0_new, x1_new, x2])):
        x1_new = np.random.normal(x1,1)
        x1_clipped = np.clip(x1_new, coil_lower, coil_upper)
    
    while not is_feasible(np.array([x0_new, x1_new, x2_new])):
        x2_new = np.random.normal(x2,1)
        x2_clipped = np.clip(x2_new, coil_num_lower, coil_num_upper)
   # print(str(x0_new) + " " + str(x1_new) + " " + str(x2_new))
    return [x0_new, x1_new, x2_new]

In [12]:
#test of distort method
start_values_array = generate_feasible_random_array()
print(start_values_array)
print(distort(start_values_array))

[ 0.08668514  0.71672592 12.85660759]
[0.08760328402380027, 0.7691950440617583, 14.480429882587739]


In [41]:
def prob(new,cur,gamma,itr):
    if new[0] < cur[0]:
        return 1
    
    delta_fx = new[0]-cur[0]
    t=gamma*itr
    
    pro= min(math.exp(-1*(delta_fx/t)),1)
    return pro

def simulated_annealing(gamma, iteration = 3000):
    
    
    
    start_values_array = generate_feasible_random_array()
    print("lets start at random " + str(start_values_array))
    best_fx = f(start_values_array)
    print("best fx so far " + str(best_fx))
    
    current_fx = best_fx
        #for every value of temperature 
        #do for number of iterations
    temp_min = 0
    for t in range(iteration):
        #     print(t)

        new_neighbours = distort(start_values_array)
        new_fx = f(new_neighbours)


#         delta_fx = new_fx[0] - current_fx[0]
#         #     print(str(new_fx) + " " + str(best_fx))
# #         print(delta_fx)
#         print(t*gamma)
#        # print(math.exp(-1*(delta_fx/(t*gamma))))
        
        pro = prob(new_fx, current_fx, gamma, t)
             
        if pro > random.random():
             current_fx = new_fx
             print("new delta e in the house by probability" + str(t) +" " + str(new_fx))
             if new_fx[0] < best_fx[0]:
                 best_fx = new_fx
             
#         if delta_fx < 0:
#             print("new delta e in the house by merit " + str(t) +" " + str(new_fx))
#             current_fx = new_fx
#             if new_fx[0] < best_fx[0]:
#                 best_fx = new_fx
#         elif (min(math.exp(-1*(delta_fx/(t*gamma))),1) > random.random()):
#             print("new delta e in the house by probability" + str(t) +" " + str(new_fx))
#             current_fx = new_fx
        #print(min(math.exp(-1*(delta_fx/(t*gamma)))
        #     print(math.exp(delta_fx/t))
    return best_fx

gamma = 0.25
simulated_annealing(gamma)
print("finished")
    

lets start at random [ 0.06643779  0.4988015  12.93315287]
best fx so far [0.032878317644242566, 18354]
new delta e in the house by probability2 [0.04314155267307597, 18357]
new delta e in the house by probability3 [0.0420522191426941, 18358]
new delta e in the house by probability4 [0.03384836294533975, 18359]
new delta e in the house by probability5 [0.03170632726009285, 18360]
new delta e in the house by probability6 [0.05387427593440153, 18361]
new delta e in the house by probability7 [0.040662250889794006, 18362]
new delta e in the house by probability8 [0.0485486490487821, 18363]
new delta e in the house by probability9 [0.04099983377083877, 18364]
new delta e in the house by probability10 [0.0315956489494888, 18365]
new delta e in the house by probability11 [0.04983330088831637, 18366]
new delta e in the house by probability12 [0.03718647606147853, 18367]
new delta e in the house by probability13 [0.054944129620731884, 18368]
new delta e in the house by probability14 [0.04499516

<ipython-input-41-01408169f209>:8: RuntimeWarning: divide by zero encountered in double_scalars
  pro= min(math.exp(-1*(delta_fx/t)),1)


new delta e in the house by probability18 [0.03649707136371674, 18373]
new delta e in the house by probability19 [0.03924241074210517, 18374]
new delta e in the house by probability20 [0.027572684245126537, 18375]
new delta e in the house by probability21 [0.045971377294868346, 18376]
new delta e in the house by probability22 [0.03748914441684832, 18377]
new delta e in the house by probability23 [0.024535441035225702, 18378]
new delta e in the house by probability24 [0.031868887917420534, 18379]
new delta e in the house by probability25 [0.03661667373602971, 18380]
new delta e in the house by probability26 [0.02149655394840224, 18381]
new delta e in the house by probability27 [0.03857938517263436, 18382]
new delta e in the house by probability28 [0.028498166096421685, 18383]
new delta e in the house by probability29 [0.036571990489489184, 18384]
new delta e in the house by probability30 [0.021290006824577705, 18385]
new delta e in the house by probability31 [0.02662024524662146, 18386]

new delta e in the house by probability137 [0.022279058533027717, 18492]
new delta e in the house by probability138 [0.0238342891890991, 18493]
new delta e in the house by probability139 [0.0241353375136327, 18494]
new delta e in the house by probability140 [0.028111376798841386, 18495]
new delta e in the house by probability141 [0.03226516334023224, 18496]
new delta e in the house by probability142 [0.02477894434432897, 18497]
new delta e in the house by probability143 [0.030335703623032104, 18498]
new delta e in the house by probability144 [0.03473735765445954, 18499]
new delta e in the house by probability145 [0.0294559533490069, 18500]
new delta e in the house by probability146 [0.05023270265896222, 18501]
new delta e in the house by probability147 [0.03404571401376391, 18502]
new delta e in the house by probability148 [0.020439363681930212, 18503]
new delta e in the house by probability149 [0.023049223175999913, 18504]
new delta e in the house by probability150 [0.0272467157576530

new delta e in the house by probability252 [0.03360891535668249, 18607]
new delta e in the house by probability253 [0.029368745076003167, 18608]
new delta e in the house by probability254 [0.0379817870264109, 18609]
new delta e in the house by probability255 [0.02301705327645474, 18610]
new delta e in the house by probability256 [0.054882927592720204, 18611]
new delta e in the house by probability257 [0.030378325766121768, 18612]
new delta e in the house by probability258 [0.046676439843855395, 18613]
new delta e in the house by probability259 [0.02850322863222272, 18614]
new delta e in the house by probability260 [0.03529408747803493, 18615]
new delta e in the house by probability261 [0.027298659524194177, 18616]
new delta e in the house by probability262 [0.029852263439727997, 18617]
new delta e in the house by probability263 [0.0477933145426933, 18618]
new delta e in the house by probability264 [0.019713206989108122, 18619]
new delta e in the house by probability265 [0.0461264335225

new delta e in the house by probability377 [0.05210174206608703, 18732]
new delta e in the house by probability378 [0.045107197020735575, 18733]
new delta e in the house by probability379 [0.035181339220182496, 18734]
new delta e in the house by probability380 [0.027765521992784613, 18735]
new delta e in the house by probability381 [0.03518256213473246, 18736]
new delta e in the house by probability382 [0.05314129534983797, 18737]
new delta e in the house by probability383 [0.03560089952213615, 18738]
new delta e in the house by probability384 [0.03314301379272023, 18739]
new delta e in the house by probability385 [0.045518545356110544, 18740]
new delta e in the house by probability386 [0.02398668283079252, 18741]
new delta e in the house by probability387 [0.03368719567941705, 18742]
new delta e in the house by probability388 [0.041700939327905834, 18743]
new delta e in the house by probability389 [0.045755135781780476, 18744]
new delta e in the house by probability390 [0.042938154806

new delta e in the house by probability500 [0.02941556903587534, 18855]
new delta e in the house by probability501 [0.03405491743583615, 18856]
new delta e in the house by probability502 [0.03783876717799341, 18857]
new delta e in the house by probability503 [0.035860733013077524, 18858]
new delta e in the house by probability504 [0.041464010604731684, 18859]
new delta e in the house by probability505 [0.031379841223335835, 18860]
new delta e in the house by probability506 [0.02743880885985404, 18861]
new delta e in the house by probability507 [0.04830811733873597, 18862]
new delta e in the house by probability508 [0.024437631369843043, 18863]
new delta e in the house by probability509 [0.04820644616497217, 18864]
new delta e in the house by probability510 [0.02311570259826307, 18865]
new delta e in the house by probability511 [0.022865884847666038, 18866]
new delta e in the house by probability512 [0.046997759375736456, 18867]
new delta e in the house by probability513 [0.034903933014

KeyboardInterrupt: 

In [ ]:
simulated_annealing()

# Task 4 - Repeations

In [42]:
#task 4 random search

lowest_fx = 1000
highest_fx = 0

best_fx_array = np.array([])
best_x_values = np.array([])

for x in range(0, 21):
    #random search
    best_x = generate_feasible_random_array()
    best_fx = f(best_x)

    for x in range(best_fx[1], (best_fx[1]+3000)):
        best_x = random_search(best_x)
        
    best_fx_array = np.append(best_fx_array, best_fx)   
    best_x_values = np.append(best_x_values, best_x)                 
    
    print(best_fx[0])
    print(lowest_fx)
    
    if (float(best_fx[0]) > highest_fx):
        highest_fx = best_fx[0]
    
    if (float(best_fx[0]) < lowest_fx):
        lowest_fx = best_fx[0]
    

print(best_fx_array)
print(best_x_values)
    
print(lowest_fx)
print(highest_fx)
    

TypeError: random_search() takes from 0 to 1 positional arguments but 2 were given

In [ ]:
#task 4 simulated annealing

lowest_fx_sa = 1000
highest_fx_sa = 0

best_fx_array_sa = np.array([])

for x in range(0, 21):
    best_fx_sa = simulated_annealing()
    
    best_fx_array_sa = np.append(best_fx_array_sa, best_fx_sa)             
    
    if (float(best_fx_sa[0]) > highest_fx_sa):
        highest_fx_sa = best_fx_sa[0]
    
    if (float(best_fx_sa[0]) < lowest_fx_sa):
        lowest_fx_sa = best_fx_sa[0]

print(best_fx_array_sa)
print(best_fx_sa)
    
print(lowest_fx_sa)
print(highest_fx_sa)

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import tkinter

# Fixing random state for reproducibility

random_data = best_fx_array
sa_data = best_fx_array_sa
data = [random_data, sa_data]

fig = plt.figure(figsize =(5, 5)) 

# Creating axes instance 
ax = fig.add_subplot() #axis
  
# Creating plot 
bp = ax.boxplot(data) 
  
# show plot 
plt.show() 



# Performances

Simmulated annealing produces worse results 

In [ ]:
print(best_fx_array)
print(best_fx_array_sa)